## 完整 py 檔見 `flask_api/app.py`

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
from pymongo import MongoClient
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_restplus import Api,Resource,fields,reqparse

# self-definition package.module
# del myfun

In [2]:
path_root = os.path.join(os.getcwd(), "../")
path_key = os.path.join(path_root, './key/key.txt')
# path_result = os.path.join(path_root, "./result")
# json_contact = os.path.join(path_result, "./contact_all.json")

# database credential
conn_string = open(path_key, 'r').read()

## test data

In [115]:
'''
- (1,2,3) = (限男生,限女生,男女皆可(不限))
- python: None, json: null
'''

# search = {
#     "shape": 1,
#     "kind": 1,
#     "renter": {
#         "renter_role": "仲介",
#         "renter_fname": "葉" ,
#         "renter_title": "先生"
#     },
#     "sex": 3, 
#     "contact": "0960803036",
#     "regionid": 1
# }

# search = {
#     "shape": 1,
#     "kind": 4,
#     "renter": {
#         "renter_role": "屋主",
#         "renter_fname": "吳" ,
#         "renter_title": "小姐"
#     },
#     "sex": 2, 
#     "contact": None,
#     "regionid": None
# }

search = {
    "shape": 1,
    "kind": None,
    "renter": {
        "renter_role": None,
        "renter_fname": None ,
        "renter_title": None
    },
    "sex": 0, 
    "contact": None,
    "regionid": None
}

# search = {
#     "renter": {
#         "renter_role": "屋主",
#         "renter_fname": "吳" ,
#         "renter_title": "小姐"
#     },
# }

In [116]:
search

{'shape': 1,
 'kind': None,
 'renter': {'renter_role': None, 'renter_fname': None, 'renter_title': None},
 'sex': 0,
 'contact': None,
 'regionid': None}

## function:search 

In [117]:
def post_json(raw_input):
    raw_input['renter.renter_role'] = raw_input["renter"]["renter_role"]
    raw_input['renter.renter_fname'] = raw_input["renter"]["renter_fname"]
    raw_input['renter.renter_title'] = raw_input["renter"]["renter_title"]
    raw_input.pop("renter")
    
    if(['sex']==3): raw_input['sex'] = None
        
    raw_dict = {k:v for k,v in raw_input.items() if v is not None}
    dictlist = []
    for key, value in raw_dict.items():
        temp = {key: value}
        dictlist.append(temp)
    return dictlist

In [118]:
a = search.copy()
b = post_json(a)
print(b)

[{'shape': 1}, {'sex': 0}]


## test search

In [119]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']
collection = db['cathay_search']
# collection.find_one({})

In [120]:
# query
results = collection.find({"$and": b}, {"_id":0})
result = []
for x in results:
    result.append(x)

In [121]:
re_post_id = []
for x in result:
    re_post_id = re_post_id + [{'post_id':x['post_id']}]

In [122]:
collection = db['meta_data']
results = collection.find({"$or": re_post_id}, {"_id":0})

In [123]:
result = []
for x in results:
    result.append(x)

OperationFailure: $and/$or/$nor must be a nonempty array, full error: {'operationTime': Timestamp(1624254147, 13), 'ok': 0.0, 'errmsg': '$and/$or/$nor must be a nonempty array', 'code': 2, 'codeName': 'BadValue', '$clusterTime': {'clusterTime': Timestamp(1624254147, 14), 'signature': {'hash': b'#\x18gC\xed\x02\xcf\xaf6f\xfbJZ\xbe\x9a\xca\x172\xbf\xcd', 'keyId': 6960683841124040705}}}

In [124]:
len(result)
# result

0

### close connection

In [28]:
cluster.close()